In [1]:
#pip install gurobipy==10

In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [3]:
# --------------- User-defined Parameters --------------- #
# Define your parameters here, for example:
M = 100000  # Maximum meals delivered by a bank in a two-week period.
D = 75  # Maximum distance a person can travel from their home to a pantry in miles.
P_min = 0.15  # Minimum percent of the FI population that must be served.
Budget = 5000000  # Total budget for the network.
Cost_per_meal_per_mile = 0.05  # Average cost per mile to transport food from a bank to a pantry.
Meals_Per_Person = 20

In [4]:
# Read the input data from the CSV files
banks_df = pd.read_csv('MSBankLatLong.csv')
pantries_df = pd.read_csv('MSPantryLatLong.csv')
demand_df = pd.read_csv('MSDemandLatLong.csv')
bank_to_pantry_distance_df = pd.read_csv('BankToPantryDistanceMatrix.csv', index_col=0)
pantry_to_demand_distance_df = pd.read_csv('PantryToDemandDistanceMatrix.csv', index_col=0)

In [5]:
#Debugging 

# Check for matching column names in the distance DataFrame and the county names
print(set(pantry_to_demand_distance_df.columns) == set(demand_df['County']))

# Try accessing a specific element
example_pantry = pantries_df['Name'].iloc[50]
example_county = demand_df['County'].iloc[70]
print("Accessing distance for:", example_pantry, "and", example_county)
print(pantry_to_demand_distance_df.loc[example_pantry, example_county])

# Check data types after conversion to numeric
print(pantry_to_demand_distance_df.dtypes)
print(pantry_to_demand_distance_df.loc["Society of St. Vincent de Paul and Nativity of the Blessed Virgin Mary Conference", "Tishomingo County"])

True
Accessing distance for: Society of St. Vincent de Paul and Nativity of the Blessed Virgin Mary Conference and Tishomingo County
336.3782818
Adams County        float64
Alcorn County       float64
Amite County        float64
Attala County       float64
Benton County       float64
                     ...   
Webster County      float64
Wilkinson County    float64
Winston County      float64
Yalobusha County    float64
Yazoo County        float64
Length: 82, dtype: object
336.3782818


In [6]:
#More debugging
# Ensure that the values are numeric and check for missing values
print(pantry_to_demand_distance_df.isnull().any().any())

# Verify that D is a numeric value
print("D:", D)

# Try accessing a specific element
example_pantry = pantries_df['Name'].iloc[0]
example_county = demand_df['County'].iloc[0]
print("Accessing distance for:", example_pantry, "and", example_county)
print(pantry_to_demand_distance_df.loc[example_pantry, example_county])


False
D: 75
Accessing distance for: Stewpot and Adams County
7.191544281


In [7]:
FI_population_df = demand_df[['County', 'FI Population 2021']].set_index('County')
#FI_population_df
FI_population_df.loc["Adams County", "FI Population 2021"]

'5,530'

In [8]:
# Initialize the model
model = gp.Model("Food_Distribution_Optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-01


In [9]:
# Decision Variables

# x_bp: Amount of food delivered from bank b to pantry p.
x_bp = model.addVars(banks_df['Name'], pantries_df['Name'], vtype=GRB.INTEGER, name="x_bp")


# y_pc: Amount of FI persons fed from pantry p to demand node c.
y_pc = model.addVars(pantries_df["Name"], demand_df["County"], vtype=GRB.INTEGER, name="y_pc")

In [10]:
# Constraints
# Bank delivery constraint
model.addConstrs((gp.quicksum(x_bp[b, p] for p in pantries_df['Name']) <= M for b in banks_df['Name']), name='bank_delivery')

# Distance constraint for pantries
model.addConstrs((pantry_to_demand_distance_df.loc[p, c]*y_pc[p,c] <= D*y_pc[p,c] for p in pantries_df['Name'] for c in demand_df['County']), name='pantry_distance')

# P_min constraint: Each county must have at least P_min percent of its food insecure people fed
FI_population_df = demand_df[['County', 'FI Population 2021']].set_index('County')
for c in demand_df['County']:
    total_fi_population = FI_population_df.loc[c, 'FI Population 2021']
    total_fi_population = int(total_fi_population.replace(',', ''))  
    model.addConstr(gp.quicksum(y_pc[p, c] for p in pantries_df['Name']) >= P_min * total_fi_population, name=f'P_min_constraint_{c}')

# Budget constraint
total_transport_cost = gp.quicksum(x_bp[b, p] * bank_to_pantry_distance_df.loc[b, p] * Cost_per_meal_per_mile 
                                   for b in banks_df['Name'] for p in pantries_df['Name'])
meal_cost_df = demand_df[['County', '2021 Cost Per Meal']].set_index('County')
total_procurement_cost = gp.quicksum(y_pc[p, c] * meal_cost_df.loc[c, '2021 Cost Per Meal'] 
                                     for p in pantries_df['Name'] for c in demand_df['County'])

total_cost = total_transport_cost + total_procurement_cost
model.addConstr(total_cost <= Budget, name='budget')

# Meals per person constraint
for p in pantries_df['Name']:
    for c in demand_df['County']:
        model.addConstr(y_pc[p, c] * Meals_Per_Person <= x_bp.sum('*', p), name=f'meals_per_person_constraint_{p}_{c}')

# Supply and demand constraint
model.addConstrs((gp.quicksum(y_pc[p, c] for c in demand_df['County']) <= gp.quicksum(x_bp[b, p] for b in banks_df['Name']) for p in pantries_df['Name']), name='supply_demand')

# Demand fulfillment constraint
model.addConstrs((y_pc[p, c] <= M for p in pantries_df['Name'] for c in demand_df['County']), name='demand_fulfillment')

# Non-negativity constraint
model.addConstrs((x_bp[b, p] >= 0 for b in banks_df['Name'] for p in pantries_df['Name']), name='non-negativity_x')
model.addConstrs((y_pc[p, c] >= 0 for p in pantries_df['Name'] for c in demand_df['County']), name='non-negativity_y')


{('Stewpot', 'Adams County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Alcorn County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Amite County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Attala County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Benton County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Bolivar County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Calhoun County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Carroll County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Chickasaw County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Choctaw County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Claiborne County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Clarke County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Clay County'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Coahoma County'): <gurobi.Constr *Awaiting M

In [11]:
# Objective Function
model.setObjective(gp.quicksum(y_pc[p, c] for p in pantries_df['Name'] for c in demand_df["County"]), GRB.MAXIMIZE)


In [12]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 73790 rows, 18870 columns and 184704 nonzeros
Model fingerprint: 0xe3c926f1
Variable types: 0 continuous, 18870 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 5e+06]
Presolve removed 70280 rows and 15002 columns
Presolve time: 0.15s
Presolved: 3510 rows, 3868 columns, 24412 nonzeros
Variable types: 0 continuous, 3868 integer (0 binary)

Root relaxation: objective 2.574760e+05, 5386 iterations, 0.10 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    257476.00000 257476.000  0.00%     -    0s

Exp

In [13]:
# Extract and print the results
solution_x_bp = model.getAttr('X', x_bp)
solution_y_pc = model.getAttr('X', y_pc)

for b, p in solution_x_bp.keys():
    if solution_x_bp[b, p] > 0:
        print(f"Bank {b} delivers {solution_x_bp[b, p]} meals to Pantry {p}.")


Bank Mississippi Food Network delivers 29000.0 meals to Pantry The Pantry.
Bank Mississippi Food Network delivers 18780.0 meals to Pantry FAITH Food Pantry.
Bank Mississippi Food Network delivers 16600.0 meals to Pantry Sam’s Diner & Food Pantry.
Bank Mississippi Food Network delivers 1620.0 meals to Pantry Ephesus Baptist Church.
Bank Mississippi Food Network delivers 4500.0 meals to Pantry Anguilla Methodist Church.
Bank Mississippi Food Network delivers 29500.0 meals to Pantry Simpson Baptist Association.
Bank Mid South Food Bank delivers 38060.0 meals to Pantry Edwards Street Fellowship Center Food Pantry.
Bank Mid South Food Bank delivers 13980.0 meals to Pantry Greenwood Interfaith Ministries Community Kitchen.
Bank Mid South Food Bank delivers 4800.0 meals to Pantry Mission Hope.
Bank Mid South Food Bank delivers 33500.0 meals to Pantry Our Daily Bread Food Pantry Wiggins.
Bank Mid South Food Bank delivers 9660.0 meals to Pantry Storehouse Community Food Pantry.
Bank Feeding The

In [14]:
for p, c in solution_y_pc.keys():
    if solution_y_pc[p, c] > 0:
        print(f"Pantry {p} feeds {solution_y_pc[p, c]} FI persons in County {c}.")

Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Covington County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Forrest County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County George County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Greene County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Hancock County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Harrison County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Jasper County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Jefferson Davis County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons in County Jones County.
Pantry Edwards Street Fellowship Center Food Pantry feeds 1903.0 FI persons 